In [22]:
import nibabel as nib
from PIL import Image
import numpy as np

# 1. 读取.nii文件
nii_img = nib.load('s3_segmented.nii')
nii_data = nii_img.get_fdata()
nii_data = np.array(nii_data).T

# 2. 读取.png图像
png_img = Image.open('s3.png')

# 3. 确保.nii数据和.png图像尺寸一致
# 取.nii数据的中间切片（假设是3D数据）
if nii_data.ndim == 3:
    middle_slice = nii_data.shape[2] // 2
    nii_data = nii_data[:, :, middle_slice]
# 确保是2D数据
assert nii_data.ndim == 2, "nii_data must be 2D after slicing"

# 将.nii_data转换为0和255的uint8数组
nii_data = (nii_data > 0).astype(np.uint8) * 255

# 创建PIL图像
nii_pil = Image.fromarray(nii_data)

# 调整尺寸到png_img的尺寸
nii_pil = nii_pil.resize(png_img.size, resample=Image.NEAREST)

# 将nii_pil转换为numpy数组
nii_array = np.array(nii_pil)

# 4. 创建绿色半透明图层
green_layer = np.zeros((nii_pil.height, nii_pil.width, 4), dtype=np.uint8)
green_layer[:, :, 1] = nii_array  # 绿色通道，255表示绿色
green_layer[:, :, 3] = np.where(nii_array > 0, 64, 0)  # alpha通道，128表示半透明

# 转换回PIL图像
green_image = Image.fromarray(green_layer)

# 5. 将绿色半透明图层叠加到背景图像上
png_img = png_img.convert("RGBA")
result = Image.alpha_composite(png_img, green_image)

# 6. 保存结果图像
result.save(r'C:\Users\25167\Desktop\123\s3Layer.png')
